## Linear Programming: Feasible and Non-Feasible Starting Points for Problems in Two and Ten Variables

### Theoretical Foundations

Linear Programming (LP) is a method to achieve the best outcome in a mathematical model whose requirements are represented by linear relationships. It is widely used in various fields such as economics, engineering, and military applications to maximize or minimize a linear objective function, subject to a set of linear inequality or equality constraints.

#### Standard Form of Linear Programming

A typical linear programming problem can be expressed in the following standard form:

- **Objective**: Minimize $ c^T x $
- **Subject to**:
  - $ A x \leq b $
  - $ x \geq 0 $

Where:
- $ x $ is the vector of decision variables.
- $ c $ is the vector of coefficients in the objective function.
- $ A $ is the matrix of coefficients in the constraints.
- $ b $ is the vector of the right-hand side values in the constraints.

### The Simplex Method

The Simplex Method, developed by George Dantzig in 1947, is a popular algorithm for solving LP problems. The method operates on linear programs in the canonical form and iteratively moves along the edges of the feasible region to find the optimal solution.

#### Key Steps in the Simplex Method

1. **Initialization**: Start from an initial feasible solution.
2. **Pivot Selection**: Determine the entering and leaving variables to move to an adjacent vertex.
3. **Iteration**: Perform the pivot operation to update the solution.
4. **Termination**: The algorithm terminates when the optimal solution is reached or if the LP problem is determined to be unbounded or infeasible.

### Feasibility and Auxiliary Problem

In practice, finding an initial feasible solution can be challenging. An auxiliary problem (also known as Phase I of the Simplex Method) is often introduced to handle this:

1. **Auxiliary Problem Setup**: Construct an auxiliary LP problem that always has a feasible solution.
2. **Solving the Auxiliary Problem**: Use the Simplex Method to find a feasible starting point for the original problem.
3. **Transition to the Original Problem**: If a feasible solution to the auxiliary problem exists, use it as the starting point for solving the original problem.

### Implementation Details

The implementation comprises several key functions: `find_feasible_start`, `simplex_method`, `simplex_method_with_feasibility_check`, and `simplex_with_custom_start`.

#### `find_feasible_start`

This function constructs and solves an auxiliary problem to find a feasible starting point for the original LP problem. The auxiliary problem is created by appending slack variables to the constraints and forming a new objective function to minimize the sum of these slack variables.

#### `simplex_method`

This function implements the core Simplex Method:
- **Tableau Construction**: The initial simplex tableau is constructed.
- **Pivot Operations**: Iteratively perform pivot operations based on the pivot column and row selection.
- **Solution Extraction**: Extract the optimal solution from the final tableau.

#### `simplex_method_with_feasibility_check`

This function integrates the feasibility check:
- **Feasibility Check**: Verify if the initial point is feasible.
- **Auxiliary Problem Solution**: If not feasible, solve the auxiliary problem to find a feasible starting point.
- **Main Simplex Method**: Proceed with the Simplex Method using the feasible starting point.

#### `simplex_with_custom_start`

This function allows solving the LP problem from a custom starting point:
- **Feasibility Check**: Verify if the custom starting point is feasible.
- **Auxiliary Problem Solution**: If not feasible, find a feasible starting point.
- **Main Simplex Method**: Solve the LP problem using the Simplex Method.

### Test Cases

The implementation is tested on multiple LP problems, both with feasible and non-feasible starting points:
- **Two-Variable Problems**: Five different LP problems are defined and solved.
- **Ten-Variable Problems**: Five different LP problems are defined and solved.

Each test case demonstrates the robustness and versatility of the Simplex Method implementation in handling various sizes and complexities of LP problems.

### Conclusion

The presented implementation effectively demonstrates the application of the Simplex Method to solve LP problems with varying complexities. By incorporating a feasibility check and solving an auxiliary problem, the implementation ensures that both feasible and non-feasible starting points are handled correctly, thereby showcasing the practical utility of the Simplex Method in real-world applications.

---

This preamble provides a comprehensive overview of the theoretical foundations, detailed explanations of the algorithm implementations, and a summary of the test cases used to validate the implementation.

### All-in-One Implementation

In [ ]:
import numpy as np
import pandas as pd
import time

# Define the functions
def find_feasible_start(A, b):
    m, n = A.shape
    A_aux = np.hstack((A, np.eye(m)))
    c_aux = np.hstack((np.zeros(n), np.ones(m)))
    x0_aux = np.zeros(n + m)
    b_aux = b

    feasible_solution, _, _, _ = simplex_method(A_aux, b_aux, c_aux)
    if np.any(feasible_solution[n:] > 1e-5):  # Use a tolerance for numerical stability
        raise ValueError("No feasible solution exists.")

    return feasible_solution[:n]

def simplex_method(A, b, c, tolerance=1e-5):
    m, n = A.shape
    tableau = np.hstack((A, np.eye(m), b.reshape(-1, 1)))
    tableau = np.vstack((tableau, np.hstack((c, np.zeros(m + 1)))))  # No need to negate c

    basis = list(range(n, n + m))

    iterations = 0
    while True:
        if np.all(tableau[-1, :-1] >= -tolerance):
            final_cost = tableau[-1, -1]
            stopping_reason = f"Optimality (cost: {final_cost})"
            break
        pivot_col = np.argmin(tableau[-1, :-1])
        if all(tableau[:-1, pivot_col] <= 0):
            stopping_reason = f"Unbounded (pivot col: {pivot_col})"
            break
        with np.errstate(divide='ignore', invalid='ignore'):
            ratios = np.where(tableau[:-1, pivot_col] > 0, tableau[:-1, -1] / tableau[:-1, pivot_col], np.inf)
        pivot_row = ratios.argmin()
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element
        for row in range(len(tableau)):
            if row != pivot_row:
                tableau[row, :] -= tableau[row, pivot_col] * tableau[pivot_row, :]
        basis[pivot_row] = pivot_col
        iterations += 1

    solution = np.zeros(n)
    for i in range(m):
        if basis[i] < n:
            solution[basis[i]] = tableau[i, -1]

    return solution, iterations, stopping_reason, tableau

def simplex_method_with_feasibility_check(A, b, c, tolerance=1e-5):
    start_time = time.time()
    x0 = np.zeros(A.shape[1])  # Default starting point
    if not np.all(np.dot(A, x0) <= b):
        x0 = find_feasible_start(A, b)

    solution, iterations, stopping_reason, tableau = simplex_method(A, b, c, tolerance)
    end_time = time.time()
    running_time = end_time - start_time

    # Calculate the error estimation
    error_estimation = np.linalg.norm(np.dot(A, solution) - b)

    return solution, iterations, stopping_reason, error_estimation, running_time

def simplex_with_custom_start(A, b, c, x0, tolerance=1e-5):
    start_time = time.time()
    if not np.all(np.dot(A, x0) <= b):
        x0 = find_feasible_start(A, b)

    solution, iterations, stopping_reason, tableau = simplex_method(A, b, c, tolerance)
    end_time = time.time()
    running_time = end_time - start_time

    # Calculate the error estimation
    error_estimation = np.linalg.norm(np.dot(A, solution) - b)

    return solution, iterations, stopping_reason, error_estimation, running_time

# Test cases for two-variable problems
lp_problems_2_variables = [
    {
        'A': np.array([[1, 2], [1, -1], [-1, 1]]),
        'b': np.array([4, 1, 1]),
        'c': np.array([-1, -2])
    },
    {
        'A': np.array([[2, 1], [1, 2], [-1, -1]]),
        'b': np.array([6, 6, -2]),
        'c': np.array([-3, -2])
    },
    {
        'A': np.array([[1, 1], [2, 3], [-1, -1]]),
        'b': np.array([5, 12, -4]),
        'c': np.array([-4, -1])
    },
    {
        'A': np.array([[3, 1], [1, -1], [-1, 1]]),
        'b': np.array([6, 1, 2]),
        'c': np.array([-1, -3])
    },
    {
        'A': np.array([[1, 3], [1, -2], [-2, 1]]),
        'b': np.array([9, 1, 2]),
        'c': np.array([-2, -1])
    }
]

non_feasible_starts_2 = [
    np.array([3, 3]),
    np.array([4, 4]),
    np.array([3, 3]),
    np.array([5, 5]),
    np.array([2, 5])
]

# Collect results in a dataframe for two-variable problems
results_2_variables = []

for i, problem in enumerate(lp_problems_2_variables):
    # Feasible start
    result_feasible, iterations_feasible, stopping_reason_feasible, error_feasible, time_feasible = simplex_method_with_feasibility_check(
        problem['A'], problem['b'], problem['c']
    )

    # Non-feasible start
    result_non_feasible, iterations_non_feasible, stopping_reason_non_feasible, error_non_feasible, time_non_feasible = simplex_with_custom_start(
        problem['A'], problem['b'], problem['c'], non_feasible_starts_2[i]
    )

    results_2_variables.append({
        'Problem': f'Problem {i + 1}',
        'Type': 'Feasible Start',
        'Iterations': iterations_feasible,
        'Final Iterate': result_feasible,
        'Error Estimation': error_feasible,
        'Running Time': time_feasible,
        'Stopping Reason': stopping_reason_feasible
    })

    results_2_variables.append({
        'Problem': f'Problem {i + 1}',
        'Type': 'Non-Feasible Start',
        'Iterations': iterations_non_feasible,
        'Final Iterate': result_non_feasible,
        'Error Estimation': error_non_feasible,
        'Running Time': time_non_feasible,
        'Stopping Reason': stopping_reason_non_feasible
    })

df_results_2_variables = pd.DataFrame(results_2_variables)

# Display the dataframe
display("Two-Variable LP Problems Results")
display(df_results_2_variables)

# Test cases for ten-variable problems
lp_problems_10_variables = [
    {
        'A': np.array([
            [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
            [1, -1, 0, 0, 0, 1, -1, 0, 0, 1],
            [-1, 0, 1, 0, 0, -1, 0, 1, 0, 0],
            [0, 1, -1, 1, 0, 0, 1, -1, 1, 0],
            [0, 0, 0, 0, -1, 1, 1, -1, 1, 1],
            [1, 0, 0, -1, 0, 0, 1, 0, -1, 0],
            [0, 1, 0, 0, 1, 0, 0, -1, 0, 1],
            [0, 0, 1, -1, 0, 1, 0, 0, 1, 0],
            [1, 0, 0, 1, -1, 0, 0, 0, 0, -1]
        ]),
        'b': np.array([5, 8, 4, 3, 7, 6, 2, 5, 3, 4]),
        'c': np.array([-2, -1, -3, -2, -4, -1, -5, -3, -1, -2])
    },
    {
        'A': np.array([
            [2, 1, 1, 0, 0, 1, 1, 0, 0, 0],
            [0, 2, 0, 1, 1, 0, 0, 1, 1, 0],
            [1, 1, 2, 1, 0, 1, 0, 1, 0, 1],
            [0, 0, 1, 1, 2, 1, 1, 0, 1, 1],
            [1, 1, 1, 0, 1, 2, 1, 1, 0, 0],
            [1, 2, 0, 1, 1, 0, 0, 1, 1, 0],
            [1, 0, 1, 2, 0, 1, 1, 0, 0, 1],
            [0, 1, 2, 1, 1, 0, 1, 1, 0, 0],
            [1, 0, 0, 1, 2, 1, 0, 1, 1, 1],
            [2, 1, 1, 0, 1, 1, 1, 0, 0, 1]
        ]),
        'b': np.array([10, 12, 14, 16, 18, 20, 22, 24, 26, 28]),
        'c': np.array([-3, -4, -5, -1, -2, -6, -7, -2, -4, -3])
    },
    {
        'A': np.array([
            [3, 2, 1, 1, 1, 0, 0, 0, 0, 0],
            [0, 3, 0, 1, 1, 1, 1, 1, 0, 0],
            [1, 1, 3, 1, 0, 1, 1, 1, 1, 0],
            [0, 0, 1, 3, 1, 1, 1, 1, 1, 1],
            [1, 1, 1, 0, 1, 3, 1, 1, 1, 0],
            [1, 2, 0, 1, 1, 0, 0, 1, 1, 1],
            [1, 0, 1, 3, 0, 1, 1, 0, 0, 1],
            [0, 1, 3, 1, 1, 0, 1, 1, 0, 0],
            [1, 0, 0, 1, 3, 1, 0, 1, 1, 1],
            [2, 1, 1, 0, 3, 1, 1, 0, 0, 1]
        ]),
        'b': np.array([15, 20, 25, 30, 35, 40, 45, 50, 55, 60]),
        'c': np.array([-2, -1, -3, -4, -5, -6, -7, -8, -9, -10])
    },
    {
        'A': np.array([
            [4, 1, 2, 1, 0, 1, 1, 0, 0, 0],
            [0, 4, 0, 1, 1, 1, 0, 1, 1, 0],
            [2, 1, 4, 1, 0, 0, 1, 1, 1, 0],
            [0, 0, 2, 4, 1, 1, 0, 0, 1, 1],
            [1, 1, 1, 0, 2, 4, 1, 1, 1, 0],
            [1, 4, 0, 1, 1, 0, 0, 1, 1, 0],
            [2, 0, 1, 4, 0, 1, 1, 0, 0, 1],
            [0, 2, 4, 1, 1, 0, 1, 1, 0, 0],
            [2, 0, 0, 1, 4, 1, 0, 1, 1, 1],
            [4, 1, 2, 0, 1, 1, 1, 0, 0, 1]
        ]),
        'b': np.array([18, 22, 26, 30, 34, 38, 42, 46, 50, 54]),
        'c': np.array([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10])
    },
    {
        'A': np.array([
            [1, 3, 2, 1, 1, 0, 0, 0, 0, 0],
            [0, 1, 3, 2, 1, 1, 0, 0, 0, 1],
            [2, 1, 1, 3, 2, 1, 0, 0, 1, 0],
            [0, 0, 1, 2, 1, 3, 0, 0, 0, 1],
            [1, 2, 1, 0, 3, 2, 1, 0, 1, 0],
            [1, 1, 3, 0, 2, 1, 0, 1, 0, 1],
            [2, 3, 0, 1, 1, 2, 1, 0, 0, 0],
            [0, 1, 2, 1, 3, 1, 0, 0, 0, 1],
            [1, 0, 1, 2, 1, 3, 0, 1, 0, 0],
            [3, 1, 1, 2, 1, 0, 1, 0, 0, 1]
        ]),
        'b': np.array([12, 16, 20, 24, 28, 32, 36, 40, 44, 48]),
        'c': np.array([-2, -3, -1, -4, -5, -6, -7, -8, -9, -10])
    }
]

non_feasible_starts_10 = [
    np.ones(10),
    np.ones(10),
    np.ones(10),
    np.ones(10),
    np.ones(10)
]

# Collect results in a dataframe for ten-variable problems
results_10_variables = []

for i, problem in enumerate(lp_problems_10_variables):
    # Feasible start
    result_feasible_10, iterations_feasible_10, stopping_reason_feasible_10, error_feasible_10, time_feasible_10 = simplex_method_with_feasibility_check(
        problem['A'], problem['b'], problem['c']
    )

    # Non-feasible start
    result_non_feasible_10, iterations_non_feasible_10, stopping_reason_non_feasible_10, error_non_feasible_10, time_non_feasible_10 = simplex_with_custom_start(
        problem['A'], problem['b'], problem['c'], non_feasible_starts_10[i]
    )

    results_10_variables.append({
        'Problem': f'Problem {i + 1}',
        'Type': 'Feasible Start',
        'Iterations': iterations_feasible_10,
        'Final Iterate': result_feasible_10,
        'Error Estimation': error_feasible_10,
        'Running Time': time_feasible_10,
        'Stopping Reason': stopping_reason_feasible_10
    })

    results_10_variables.append({
        'Problem': f'Problem {i + 1}',
        'Type': 'Non-Feasible Start',
        'Iterations': iterations_non_feasible_10,
        'Final Iterate': result_non_feasible_10,
        'Error Estimation': error_non_feasible_10,
        'Running Time': time_non_feasible_10,
        'Stopping Reason': stopping_reason_non_feasible_10
    })

df_results_10_variables = pd.DataFrame(results_10_variables)

# Display the dataframe
display("Ten-Variable LP Problems Results")
display(df_results_10_variables)


'Two-Variable LP Problems Results'

,Problem,Type,Iterations,Final Iterate,Error Estimation,Running Time,Stopping Reason
0,Problem 1,Feasible Start,2,"[0.6666666666666666, 1.6666666666666665]",2.000000,0.000344,Optimality (cost: 4.0)
1,Problem 1,Non-Feasible Start,2,"[0.6666666666666666, 1.6666666666666665]",2.000000,0.000325,Optimality (cost: 4.0)
2,Problem 2,Feasible Start,2,"[2.0, 2.0]",2.000000,0.000591,Optimality (cost: 10.0)
3,Problem 2,Non-Feasible Start,2,"[2.0, 2.0]",2.000000,0.000373,Optimality (cost: 10.0)
4,Problem 3,Feasible Start,1,"[5.0, 0.0]",2.236068,0.000267,Optimality (cost: 20.0)
5,Problem 3,Non-Feasible Start,1,"[5.0, 0.0]",2.236068,0.000219,Optimality (cost: 20.0)
6,Problem 4,Feasible Start,2,"[1.0, 3.0]",3.000000,0.000174,Optimality (cost: 10.0)
7,Problem 4,Non-Feasible Start,2,"[1.0, 3.0]",3.000000,0.000633,Optimality (cost: 10.0)
8,Problem 5,Feasible Start,2,"[4.2, 1.6]",8.800000,0.000204,Optimality (cost: 10.0)
9,Problem 5,Non-Feasible Start,2,"[4.2, 1.6]",8.800000,0.000436,Optimality (cost: 10.0)


'Ten-Variable LP Problems Results'

,Problem,Type,Iterations,Final Iterate,Error Estimation,Running Time,Stopping Reason
0,Problem 1,Feasible Start,10,"[1.4545454545454553, 0.0, 0.27272727272727115,...",9.734067,0.000949,Optimality (cost: 54.63636363636365)
1,Problem 1,Non-Feasible Start,10,"[1.4545454545454553, 0.0, 0.27272727272727115,...",9.734067,0.000814,Optimality (cost: 54.63636363636365)
2,Problem 2,Feasible Start,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 4.0,...",24.819347,0.000358,Optimality (cost: 108.0)
3,Problem 2,Non-Feasible Start,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 4.0,...",24.819347,0.000340,Optimality (cost: 108.0)
4,Problem 3,Feasible Start,2,"[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",71.589105,0.000258,Optimality (cost: 310.0)
5,Problem 3,Non-Feasible Start,2,"[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",71.589105,0.000296,Optimality (cost: 310.0)
6,Problem 4,Feasible Start,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 20.0, 0.0,...",36.000000,0.000295,Optimality (cost: 502.0)
7,Problem 4,Non-Feasible Start,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 20.0, 0.0,...",36.000000,0.000334,Optimality (cost: 502.0)
8,Problem 5,Feasible Start,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 20.0...",54.110997,0.000532,Optimality (cost: 524.0)
9,Problem 5,Non-Feasible Start,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 20.0...",54.110997,0.000323,Optimality (cost: 524.0)
